# Comparing Huggingface Biomedical Transformer Models

In [ ]:
import numpy as np
import pandas as pd

import torch
from transformers import pipeline, AutoModel, AutoConfig, AutoTokenizer

from scipy.spatial.distance import cosine
import os

Facebook's Bio-LM Model

In [ ]:
# download is from their GitHub page
bio_lm_filepath = "/home/mila/d/david.hobson/scratch/code/modelling/huggingface_models/RoBERTa-large-PM-M3"
os.chdir(bio_lm_filepath)

config = AutoConfig.from_pretrained(
    "RoBERTa-large-PM-M3-hf",
)
model = AutoModel.from_pretrained("RoBERTa-large-PM-M3-hf", config=config)

biolm = pipeline('feature-extraction', model=model, tokenizer='roberta-large')

Microsoft's PubMedBERT

In [ ]:
pm_bert = pipeline('feature-extraction', model='microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext', framework='pt')

DMIS Disease Feature Extractor

In [ ]:
dmis = pipeline('feature-extraction', model='dmis-lab/biosyn-sapbert-bc5cdr-disease', framework='pt')

## Compare Embeddings

In [ ]:
def embedding_dist(a, b, feature_extractor):
    a_embedded = np.squeeze(feature_extractor(a)).mean(axis=0)
    b_embedded = np.squeeze(feature_extractor(b)).mean(axis=0)

    return 1 - cosine(a_embedded, b_embedded)

def display_embedding_dist(a, b):
    print(f"""
    ('{a}', '{b}'):\n
    \tBio-LM: {embedding_dist(a, b, biolm)}\n
    \tPubMedBERT: {embedding_dist(a, b, pm_bert)}\n
    \tDMIS: {embedding_dist(a, b, dmis)}
    """)
    
def embedding_df(conditions_list):
    embedding_results = []
    for c1, c2 in conditions_list:
        embeddings = {}
        embeddings['PubMedBERT'] = np.round(embedding_dist(c1, c2, pm_bert), 3)
        embeddings['Bio-LM'] = np.round(embedding_dist(c1, c2, biolm), 3)
        embeddings['DMIS'] = np.round(embedding_dist(c1, c2, dmis), 3)
        embedding_results.append(embeddings)
    return pd.DataFrame(embedding_results, index=conditions_list)

In [ ]:
same_conditions = [
    ('CVD', 'cardiovascular disease'),
    ('COPD', 'Chronic Obstructive Pulmonary Disease'),
    ("AIDS", 'HIV'),
    ('cancer', 'neoplasms'),
    ('womb cancer', 'uterine cancer'),
    ('flu', 'influenza'),
    ('COVID-19', 'coronavirus'),
]

diff_conditions =[   
    ('CVD', 'heart attack'),
    ('CVD', 'COPD'),
    ('Diabetes', 'Obesity'),
    ('Anxiety', 'Depression'),
    ('Jaw pain', 'Pink eye'),
    ("CVD", 'Nebraska'),
    ('AIDS', 'Cotopaxi'),
    ('')
]

In [ ]:
embedding_df(same_conditions)

In [ ]:
embedding_df(diff_conditions)

In [ ]:
# case sensitive, uses byte-level BPE tokenization
t = AutoTokenizer.from_pretrained("roberta-large")

In [ ]:
t("I have cancer")

In [ ]:
t.decode(t("hiv")['input_ids'])

## Cluster the embeddings

In [ ]:
from scipy.cluster.hierarchy import linkage, fcluster

In [ ]:
df_vars = pd.read_csv("~/scratch/datasets/yale_new_haven/supplementary_info/variable_descriptions.csv")

In [ ]:
disposition_var = {'disposition'}

demographic_vars = {'age', 'gender', 'ethnicity', 'race', 'lang',
       'religion', 'maritalstatus', 'employstatus', 'insurance_status'}

# department name, ESI score, arrival info, and triage vital info
triage_evaluation_vars = {'dep_name', 'esi', 'arrivalmode', 'arrivalmonth', 'arrivalday', 'arrivalhour_bin'}.union({col for col in df_vars.columns if 'triage_vital' in col})

# chief complaint info (only top 200 were included; represents >90% of the complaints)
chief_complaint_vars = {col for col in df_vars['Variable Name'] if "cc_" in col}

# medication info
medication_vars = {col for col in df_vars['Variable Name'] if 'meds_' in col}

hospital_usage_stats_vars = {'previousdispo', 'n_edvisits', 'n_admissions', 'n_surgeries'}

# prior imaging and EKG counts
# chest x-ray, echocardiogram, electrocardiogram (EKG), other x-ray, other ultra-sound, head CT, other CT, MRI, and all other imaging
imaging_ekg_vars = {'cxr_count','echo_count','ekg_count','otherxr_count', 'otherus_count', 'headct_count', 'otherct_count', 'mri_count','otherimg_count'}

# historic vitals include: systolic blood pressure, diastolic blood pressure, pulse, respiratory rate, oxygen saturation, presence of oxygen device, and temperature
historical_vital_vars = {'dbp_last',
 'dbp_max',
 'dbp_median',
 'dbp_min',
 'o2_device_last',
 'o2_device_max',
 'o2_device_median',
 'o2_device_min',
 'pulse_last',
 'pulse_max',
 'pulse_median',
 'pulse_min',
 'resp_last',
 'resp_max',
 'resp_median',
 'resp_min',
 'sbp_last',
 'sbp_max',
 'sbp_median',
 'sbp_min',
 'spo2_last',
 'spo2_max',
 'spo2_median',
 'spo2_min',
 'temp_last',
 'temp_max',
 'temp_median',
 'temp_min'}

curr = disposition_var.union(demographic_vars.union(triage_evaluation_vars.union(chief_complaint_vars.union(medication_vars.union(hospital_usage_stats_vars.union(imaging_ekg_vars.union(historical_vital_vars)))))))

# past medical history
past_medical_hist_vars = {col for col in df_vars['Variable Name'] if col not in curr and "_" not in col and 'previousdispo' not in col}

curr = curr.union(past_medical_hist_vars)

# historical labs ordered by ED (only top 150 comprising of 94% of all orders)
historical_lab_vars = {col for col in df_vars['Variable Name'] if col not in curr}

In [ ]:
def cluster(df_tmp, clustering_features, d_thres='0.1', method='complete'):
    df_tmp = df_tmp[df_tmp['Variable Name'].isin(clustering_features)][['Variable Name', 'Description']]
    embeddings = np.array([np.squeeze(feature_extraction(desc)).mean(axis=0) for desc in df_tmp['Description']])
    
    Z = linkage(embeddings, method=method, metric='cosine')
    
    clustering = fcluster(Z, d_thres, criterion='distance')
    df_tmp['Cluster'] = pd.Series(clustering, index=df_tmp.index)
    
    return df_tmp

In [ ]:
clustering_features = list(past_medical_hist_vars)

In [ ]:
df_cc = df_vars[df_vars['Variable Name'].isin(clustering_features)][['Variable Name', 'Description']]

In [ ]:
cc_embeddings = np.array([np.squeeze(feature_extraction(cc)).mean(axis=0) for cc in df_cc['Description']])

In [ ]:
Z = linkage(cc_embeddings, method='complete', metric='cosine')

In [ ]:
d_thres = 0.15

In [ ]:
clustering = fcluster(Z, d_thres, criterion='distance')

In [ ]:
df_cc['Cluster'] = pd.Series(clustering, index=df_cc.index)

In [ ]:
cluster_dict = {i: [] for i in df_cc['Cluster'].sort_values().unique()}
for i, row in df_cc.iterrows():
    cluster_dict[row['Cluster']].append(row['Description'])

In [ ]:
for i in cluster_dict:
    print(f"{i}) {cluster_dict[i]}")
    print()

In [ ]:
df = pd.read_csv("~/scratch/datasets/yale_new_haven/training_test_sets/second_split/features/regression_nn/yale_new_haven_test_features.csv")

In [ ]:
cc_vars = [col for col in df.columns if "cc_" in col]

In [ ]:
df_vars = pd.read_csv("~/scratch/datasets/yale_new_haven/supplementary_info/variable_descriptions.csv")

In [ ]:
var_desc_dict = dict()
for i, row in df_vars[['Variable Name', 'Description']][df_vars['Variable Name'].isin(cc_vars)].iterrows():
    var_desc_dict[row['Variable Name']] = row['Description']

In [ ]:
problems = df[cc_vars].T.apply(lambda x: [var_desc_dict[i] for i in x[x == 1].index])

In [ ]:
# some people have no problems... What to do about them...?
# zeros will kill all the weights I guess...

In [ ]:
problems

In [ ]:
kk = problems[problems.apply(lambda x: len(x)) > 1].index

In [ ]:
problems[kk[1]]

In [ ]:
ep = problems[:100].apply(lambda x: np.array([np.squeeze(feature_extraction(problem)).mean(axis=0) for problem in x]).mean(axis=0))

In [ ]:
# storing the embeddings of all of the complaints is going to take A LOT of memory... 
# ~200,000 x 768 floats just for the complaints (non-zero...)
np.squeeze(feature_extraction('HIV')).dtype

In [ ]:
np.array(l).mean(axis=0).shape